In [ ]:
import polars as pl
import polars_ds as pds

In [ ]:
df = pl.DataFrame({
    "x": [1,1,3,3,3,6,7,7,8,10,11,1, None],
    "y": [3,4,2,4,10,6,10,9,8,10,1,7, None]
}) # .sort("x")

In [ ]:
df.select(
    pds.kendall_tau("x", "y")
)

In [ ]:
df.select(
    pds.kendall_tau2("x", "y")
)

In [ ]:
df = pds.random_data(size=5000, n_cols = 0).select(
    pds.random_int(0, 200).alias("x"),
    pds.random_int(0, 200).alias("y"),
)
df.head()

In [ ]:
%timeit df.select(pds.kendall_tau("x", "y"))
%timeit df.select(pds.kendall_tau2("x", "y"))

In [ ]:
from scipy.stats import kendalltau

x = df["x"].to_numpy()
y = df["y"].to_numpy()

In [ ]:
%%timeit
kendalltau(x,y, nan_policy="omit")

In [ ]:
df.sort(pl.col("x").rank(method="random")).select(
    "x",
    "y",
    pl.col("y").rank(method="max").cast(pl.Float64).alias("r"),
    (-pl.col("y")).rank(method="max").cast(pl.Float64).alias("l"),
).with_columns(
    pl.col("r").diff().abs().alias("r_abs_diff"),
    (pl.col("l") * (pl.len() - pl.col("l"))).alias("l(n-l)"),
).select(
    1 - (pl.len() / 2) * (pl.col("r_abs_diff").sum() / pl.col("l(n-l)").sum())
)